# Importing the necessary libraries

In [1]:
import collections
from datetime import datetime
import numpy as np
import json
import tensorflow.strings as tf_strings
from keras.preprocessing.text import tokenizer_from_json
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, LSTM, Dropout, Activation, TimeDistributed, TextVectorization
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# Verify GPU

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15580393273827151880
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2236245607
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8544199716593428306
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


# Loading the tokenizers and pretrained model

In [3]:
## loading the model

model_hi = load_model('english_to_hindi_lstm_model')

#load Tokenizer
with open('english_tokenizer_hindi.json') as f:
    data = json.load(f)
    english_tokenizer = tokenizer_from_json(data)

with open('hindi_tokenizer.json') as f:
    data = json.load(f)
    hindi_tokenizer = tokenizer_from_json(data)

In [4]:
with open('sequence_length_hindi.json') as f:
    max_length_hindi = json.load(f)
    
def pad(x, length=None):
    return pad_sequences(x, maxlen=length, padding='post')

def translate_to_hindi(english_sentence):
    english_sentence = english_sentence.lower()
    
    english_sentence = english_sentence.replace(".", '')
    english_sentence = english_sentence.replace("?", '')
    english_sentence = english_sentence.replace("!", '')
    english_sentence = english_sentence.replace(",", '')
    
    english_sentence = english_tokenizer.texts_to_sequences([english_sentence])
    english_sentence = pad(english_sentence, max_length_hindi)
    
    english_sentence = english_sentence.reshape((-1,max_length_hindi))
    
    hindi_sentence = model_hi.predict(english_sentence)[0]
    
    hindi_sentence = [np.argmax(word) for word in hindi_sentence]

    hindi_sentence = hindi_tokenizer.sequences_to_texts([hindi_sentence])[0]
    
    # print("hindi translation: ", hindi_sentence)
    
    return hindi_sentence

In [8]:

# Function to check if a word starts with a vowel
def starts_with_vowel(word):
    vowels = 'AEIOUaeiou'
    return word[0] in vowels

# Function to translate English word to Hindi
def translate(word):
    
    current_time = datetime.now()
    current_hour = current_time.hour
    
    if starts_with_vowel(word):
        if current_hour < 21 or current_hour > 21:
            return "This word starts with Vowels, provide some other words." 
        else:
            translation = translate_to_hindi(word)
            return translation
    else:
        translation = translate_to_hindi(word)
        return translation


# Example usage
english_word = input("Enter an English word: ")

translated_word = translate(english_word)
print(translated_word)


1/1 [==============================] - 0s 120ms/step
वह के है।
